# SQL III - View
## 1. Introduction
### What is a View?
A view is a reusable virtual table that is created by querying existing tables. The virtual table is not saved in memory. Instead, the query used for the view is saved with the name. Each time the view is used, a new query of that view is being executed. 

---

### Why Are Views Important?
- **Privacy**: views can limit the information shared by taking a subset of information from full tables
- **Reusability**: can make repetitive queries easier to write
- **Generalization**: can combine different sub categories into a larger category

---

### Setup

In [8]:
# Imports
from tabulate import tabulate
import psycopg2
import time

# Database connection parameters
dbname = 'banking_db'
user = 'postgres'
password = 'postgres'
host = 'postgres_db'  # This should be the service name defined in docker-compose.yml
port = '5432'  

# Establish connection to the PostgreSQL database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Create a cursor object to interact with the database
cur = conn.cursor()

In [9]:
# Create tables

cur.execute("""
    CREATE TABLE agents
    (	
        AGENT_CODE char(6) NOT NULL PRIMARY KEY, 
    	AGENT_NAME char(40), 
    	WORKING_AREA char(35), 
    	COMMISSION numeric(10,2), 
    	PHONE_NO char(15), 
    	COUNTRY varchar(25) 
	);
""")

cur.execute("""
    INSERT INTO agents VALUES ('A007', 'Ramasundar', 'Bangalore', '0.15', '077-25814763', '');
    INSERT INTO agents VALUES ('A003', 'Alex ', 'London', '0.13', '075-12458969', '');
    INSERT INTO agents VALUES ('A008', 'Alford', 'New York', '0.12', '044-25874365', '');
    INSERT INTO agents VALUES ('A011', 'Ravi Kumar', 'Bangalore', '0.15', '077-45625874', '');
    INSERT INTO agents VALUES ('A010', 'Santakumar', 'Chennai', '0.14', '007-22388644', '');
    INSERT INTO agents VALUES ('A012', 'Lucida', 'San Jose', '0.12', '044-52981425', '');
    INSERT INTO agents VALUES ('A005', 'Anderson', 'Brisbane', '0.13', '045-21447739', '');
    INSERT INTO agents VALUES ('A001', 'Subbarao', 'Bangalore', '0.14', '077-12346674', '');
    INSERT INTO agents VALUES ('A002', 'Mukesh', 'Mumbai', '0.11', '029-12358964', '');
    INSERT INTO agents VALUES ('A006', 'McDen', 'London', '0.15', '078-22255588', '');
    INSERT INTO agents VALUES ('A004', 'Ivan', 'Toronto', '0.15', '008-22544166', '');
    INSERT INTO agents VALUES ('A009', 'Benjamin', 'Hampshair', '0.11', '008-22536178', '');
""")

cur.execute("""
    CREATE TABLE customer
    (	
        CUST_CODE varchar(6) NOT NULL PRIMARY KEY, 
    	CUST_NAME varchar(40) NOT NULL, 
    	CUST_CITY char(35), 
    	WORKING_AREA varchar(35) NOT NULL, 
    	CUST_COUNTRY varchar(20) NOT NULL, 
    	GRADE integer, 
    	OPENING_AMT numeric(12,2) NOT NULL, 
    	RECEIVE_AMT numeric(12,2) NOT NULL, 
    	PAYMENT_AMT numeric(12,2) NOT NULL, 
    	OUTSTANDING_AMT numeric(12,2) NOT NULL, 
    	PHONE_NO varchar(17) NOT NULL, 
    	AGENT_CODE char(6) NOT NULL REFERENCES agents
    ); 
""")

cur.execute("""
    INSERT INTO customer VALUES ('C00013', 'Holmes', 'London', 'London', 'UK', '2', '6000.00', '5000.00', '7000.00', '4000.00', 'BBBBBBB', 'A003');
    INSERT INTO customer VALUES ('C00001', 'Micheal', 'New York', 'New York', 'USA', '2', '3000.00', '5000.00', '2000.00', '6000.00', 'CCCCCCC', 'A008');
    INSERT INTO customer VALUES ('C00020', 'Albert', 'New York', 'New York', 'USA', '3', '5000.00', '7000.00', '6000.00', '6000.00', 'BBBBSBB', 'A008');
    INSERT INTO customer VALUES ('C00025', 'Ravindran', 'Bangalore', 'Bangalore', 'India', '2', '5000.00', '7000.00', '4000.00', '8000.00', 'AVAVAVA', 'A011');
    INSERT INTO customer VALUES ('C00024', 'Cook', 'London', 'London', 'UK', '2', '4000.00', '9000.00', '7000.00', '6000.00', 'FSDDSDF', 'A006');
    INSERT INTO customer VALUES ('C00015', 'Stuart', 'London', 'London', 'UK', '1', '6000.00', '8000.00', '3000.00', '11000.00', 'GFSGERS', 'A003');
    INSERT INTO customer VALUES ('C00002', 'Bolt', 'New York', 'New York', 'USA', '3', '5000.00', '7000.00', '9000.00', '3000.00', 'DDNRDRH', 'A008');
    INSERT INTO customer VALUES ('C00018', 'Fleming', 'Brisbane', 'Brisbane', 'Australia', '2', '7000.00', '7000.00', '9000.00', '5000.00', 'NHBGVFC', 'A005');
    INSERT INTO customer VALUES ('C00021', 'Jacks', 'Brisbane', 'Brisbane', 'Australia', '1', '7000.00', '7000.00', '7000.00', '7000.00', 'WERTGDF', 'A005');
    INSERT INTO customer VALUES ('C00019', 'Yearannaidu', 'Chennai', 'Chennai', 'India', '1', '8000.00', '7000.00', '7000.00', '8000.00', 'ZZZZBFV', 'A010');
    INSERT INTO customer VALUES ('C00005', 'Sasikant', 'Mumbai', 'Mumbai', 'India', '1', '7000.00', '11000.00', '7000.00', '11000.00', '147-25896312', 'A002');
    INSERT INTO customer VALUES ('C00007', 'Ramanathan', 'Chennai', 'Chennai', 'India', '1', '7000.00', '11000.00', '9000.00', '9000.00', 'GHRDWSD', 'A010');
    INSERT INTO customer VALUES ('C00022', 'Avinash', 'Mumbai', 'Mumbai', 'India', '2', '7000.00', '11000.00', '9000.00', '9000.00', '113-12345678','A002');
    INSERT INTO customer VALUES ('C00004', 'Winston', 'Brisbane', 'Brisbane', 'Australia', '1', '5000.00', '8000.00', '7000.00', '6000.00', 'AAAAAAA', 'A005');
    INSERT INTO customer VALUES ('C00023', 'Karl', 'London', 'London', 'UK', '0', '4000.00', '6000.00', '7000.00', '3000.00', 'AAAABAA', 'A006');
    INSERT INTO customer VALUES ('C00006', 'Shilton', 'Toronto', 'Toronto', 'Canada', '1', '10000.00', '7000.00', '6000.00', '11000.00', 'DDDDDDD', 'A004');
    INSERT INTO customer VALUES ('C00010', 'Charles', 'Hampshair', 'Hampshair', 'UK', '3', '6000.00', '4000.00', '5000.00', '5000.00', 'MMMMMMM', 'A009');
    INSERT INTO customer VALUES ('C00017', 'Srinivas', 'Bangalore', 'Bangalore', 'India', '2', '8000.00', '4000.00', '3000.00', '9000.00', 'AAAAAAB', 'A007');
    INSERT INTO customer VALUES ('C00012', 'Steven', 'San Jose', 'San Jose', 'USA', '1', '5000.00', '7000.00', '9000.00', '3000.00', 'KRFYGJK', 'A012');
    INSERT INTO customer VALUES ('C00008', 'Karolina', 'Toronto', 'Toronto', 'Canada', '1', '7000.00', '7000.00', '9000.00', '5000.00', 'HJKORED', 'A004');
    INSERT INTO customer VALUES ('C00003', 'Martin', 'Toronto', 'Toronto', 'Canada', '2', '8000.00', '7000.00', '7000.00', '8000.00', 'MJYURFD', 'A004');
    INSERT INTO customer VALUES ('C00009', 'Ramesh', 'Mumbai', 'Mumbai', 'India', '3', '8000.00', '7000.00', '3000.00', '12000.00', 'Phone No', 'A002');
    INSERT INTO customer VALUES ('C00014', 'Rangarappa', 'Bangalore', 'Bangalore', 'India', '2', '8000.00', '11000.00', '7000.00', '12000.00', 'AAAATGF', 'A001');
    INSERT INTO customer VALUES ('C00016', 'Venkatpati', 'Bangalore', 'Bangalore', 'India', '2', '8000.00', '11000.00', '7000.00', '12000.00', 'JRTVFDD', 'A007');
    INSERT INTO customer VALUES ('C00011', 'Sundariya', 'Chennai', 'Chennai', 'India', '3', '7000.00', '11000.00', '7000.00', '11000.00', 'PPHGRTS', 'A010');
""")

cur.execute("""
    CREATE TABLE contractors
    (	
        CONTRACTOR_CODE char(6) NOT NULL PRIMARY KEY, 
    	CONTRACTOR_NAME char(40), 
    	WORKING_AREA char(35), 
    	COMMISSION numeric(10,2), 
    	PHONE_NO char(15), 
    	COUNTRY varchar(25) 
	);
""")

cur.execute("""
    INSERT INTO contractors VALUES ('C001', 'John', 'New York', '0.17', '077-11124769', '');
    INSERT INTO contractors VALUES ('C002', 'Sarah ', 'San Francisco', '0.18', '032-89790125', '');
    INSERT INTO contractors VALUES ('C003', 'Liam', 'Dublin', '0.14', '011-33447076', '');
    INSERT INTO contractors VALUES ('C004', 'Richard', 'Vancouver', '0.15', '008-44720826', '');
    INSERT INTO contractors VALUES ('C005', 'Whitaker', 'New York', '0.18', '012-58859201', '');
    INSERT INTO contractors VALUES ('C006', 'Lucia', 'San Jose', '0.15', '032-81027792', '');
""")

conn.commit()

In [10]:
## Print table

# Query to fetch records from the accounts table
cur.execute("SELECT * FROM agents")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+--------------+--------------+----------------+--------------+--------------+-----------+
| agent_code   | agent_name   | working_area   |   commission | phone_no     | country   |
+==============+==============+================+==============+==============+===========+
| A007         | Ramasundar   | Bangalore      |         0.15 | 077-25814763 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+
| A003         | Alex         | London         |         0.13 | 075-12458969 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+
| A008         | Alford       | New York       |         0.12 | 044-25874365 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+
| A011         | Ravi Kumar   | Bangalore      |         0.15 | 077-45625874 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+

## 2. Creating Views
### 2.1 Privacy 
#### I want to give access to all employees in my company the name, working area, and phone number of all agents. 
#### However I don't want to share the agent code and commission. We can achieve this by creating a view. 

In [11]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        CREATE VIEW public_agents AS 
        SELECT agent_name, working_area, phone_no
        FROM agents 
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [12]:
# Query to fetch records from the accounts table
cur.execute("SELECT * FROM public_agents")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+--------------+----------------+--------------+
| agent_name   | working_area   | phone_no     |
+==============+================+==============+
| Ramasundar   | Bangalore      | 077-25814763 |
+--------------+----------------+--------------+
| Alex         | London         | 075-12458969 |
+--------------+----------------+--------------+
| Alford       | New York       | 044-25874365 |
+--------------+----------------+--------------+
| Ravi Kumar   | Bangalore      | 077-45625874 |
+--------------+----------------+--------------+
| Santakumar   | Chennai        | 007-22388644 |
+--------------+----------------+--------------+
| Lucida       | San Jose       | 044-52981425 |
+--------------+----------------+--------------+
| Anderson     | Brisbane       | 045-21447739 |
+--------------+----------------+--------------+
| Subbarao     | Bangalore      | 077-12346674 |
+--------------+----------------+--------------+
| Mukesh       | Mumbai         | 029-12358964 |
+--------------+----

#### Employees can now be given access to the view **public_employees** without sharing sensitive information. 
--- 
### 2.2 Reusability 
#### I regularly have to make queries on the customers living in the US and India. To do this, I make a custom view that I can reuse for further queries. 

In [13]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

# Resets the view
# cur.execute("DROP VIEW customers_USA_india")

try: 
    # Create view
    cur.execute("""
        CREATE VIEW customers_USA_india AS 
        SELECT cust_code, cust_name, cust_city, cust_country, grade, outstanding_amt, agent_code
        FROM customer 
        WHERE CUST_COUNTRY = 'USA' OR CUST_COUNTRY = 'India'
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [14]:
# Query to fetch records
cur.execute("SELECT * FROM customers_USA_india")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| cust_code   | cust_name   | cust_city   | cust_country   |   grade |   outstanding_amt | agent_code   |
+=============+=============+=============+================+=========+===================+==============+
| C00001      | Micheal     | New York    | USA            |       2 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00020      | Albert      | New York    | USA            |       3 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00025      | Ravindran   | Bangalore   | India          |       2 |              8000 | A011         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00002      | Bolt        | New York    | US

#### Now that I have created my view, I can query it just like any table. For example, let's say that I want to find customers with a grade greater or equal to 2. 

In [15]:
# Query to fetch records
cur.execute("""
    SELECT * 
    FROM customers_USA_india
    WHERE grade >= 2
""")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| cust_code   | cust_name   | cust_city   | cust_country   |   grade |   outstanding_amt | agent_code   |
+=============+=============+=============+================+=========+===================+==============+
| C00001      | Micheal     | New York    | USA            |       2 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00020      | Albert      | New York    | USA            |       3 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00025      | Ravindran   | Bangalore   | India          |       2 |              8000 | A011         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00002      | Bolt        | New York    | US

#### We can continue querying the same view of the US and India customers. This time I want to check for any customers who have an outstanding balance over 3000 and residing in New York.

In [16]:
# Query to fetch records
cur.execute("""
    SELECT * 
    FROM customers_USA_india
    WHERE outstanding_amt > 3000 AND cust_city = 'New York'
""")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| cust_code   | cust_name   | cust_city   | cust_country   |   grade |   outstanding_amt | agent_code   |
+=============+=============+=============+================+=========+===================+==============+
| C00001      | Micheal     | New York    | USA            |       2 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+
| C00020      | Albert      | New York    | USA            |       3 |              6000 | A008         |
+-------------+-------------+-------------+----------------+---------+-------------------+--------------+


### 2.3 Generalization
#### Generalization can help create views that combine two similar categories into one. In this example, we want to query a table containing both agents and contractors. To do this, we can combine the two tables into one view called **all_employees**. 

In [17]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        CREATE VIEW all_employees AS 
        SELECT *
        FROM agents
        UNION
        SELECT * 
        FROM contractors
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [18]:
# Query to fetch records 
cur.execute("SELECT * FROM all_employees")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+--------------+--------------+----------------+--------------+--------------+-----------+
| agent_code   | agent_name   | working_area   |   commission | phone_no     | country   |
+==============+==============+================+==============+==============+===========+
| C003         | Liam         | Dublin         |         0.14 | 011-33447076 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+
| A001         | Subbarao     | Bangalore      |         0.14 | 077-12346674 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+
| A005         | Anderson     | Brisbane       |         0.13 | 045-21447739 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+
| A012         | Lucida       | San Jose       |         0.12 | 044-52981425 |           |
+--------------+--------------+----------------+--------------+--------------+-----------+